In [1]:
import time
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
import re
import os

In [2]:
class Data:
    def __init__(self,name,profit,day,hours,amount,method):
        self.name= name.title()
        self.profit= profit
        self.day= pd.to_datetime(day)
        self.amount = amount
        self.method = method.title()
        self.hours = float(hours)

    def create_data_object():
        name,profit,day,hours,amount,method = input('Name of data? '), input('How much did you make? '),input('Date of profit? '),input('How many hours did this take? '),input('What was the amount that created this profit? '),input('What type of method was this? (Bossing,Slayer,Skiling, etc)')
        data = Data(name,profit,day,hours,amount,method)
        return data

    def create_dict(self):
        return dict(Name = [self.name],
             Profit = [self.profit],
             Day= [self.day],
             Amount = [self.amount],
             Method= [self.method],
             Hours = [self.hours]
            )

In [3]:
def add_item():
    data = Data.create_data_object()
    total_df.append(pd.DataFrame(data.create_dict()))
    df = pd.concat(total_df,ignore_index=True,sort=False)
    return df

def sort_item(dataframe):
    print('How would you like to search through your data?\n1.Name\n2.Profit\n3.Day\n4.Amount\n5.Method\n6.Hours')
    number = int(input('Number: '))
    sort = columns[number - 1]
    return dataframe.sort_values(by = sort)

def sort_by_date(dataframe):
    return dataframe.sort_values(by='Day', ascending = False)

def edit_item(sorted_df):
    print('How would you like to search through your data?\n1.Name\n2.Profit\n3.Day\n4.Amount\n5.Method\n6.Hours')
    number = int(input('Number: '))
    sort = columns[number - 1]
    sorted_df = sorted_df.set_index(sort)
    print(sorted_df)
    item = input('Which item do you want to edit: ')
    column = input('Which value do you want to edit: ')
    new_value = input('What is the new value: ')
    if type(new_value) == str:
        new_value = new_value.title()
    sorted_df.loc[item.title(),column.title()] = new_value
    return sorted_df.reset_index()

def delete_item(sorted_df):
    item = input('What item do you want to delete?')
    for i in range(len(columns)):
        index = sorted_df[sorted_df[columns[i]] == item.title()].index
        if len(index) > 1:
            print('Mulitple %s logs found.' % item.title())
            print(sorted_df[sorted_df[columns[i]] == item.title()])
            delete = int(input('Which entry number would you like to delete?'))
            sorted_df.drop(delete,inplace=True)
            sorted_df.reset_index(drop=True,inplace =True)
            return sorted_df
        elif len(index) == 1:
            sorted_df.drop(sorted_df[sorted_df[columns[i]] == item.title()].index,inplace=True)
            sorted_df.reset_index(drop=True,inplace =True)
            return sorted_df
        else:
            continue

def check_log(dataframe):
    print(dataframe)
    print('Do you want to sort the data?')
    answer = input("")
    if answer.lower()[0] == 'y':
        sorts = input('Whcich column do you want to sort along? (Name, Profit, Method etc): ')
        print(dataframe.sort_values(by = sorts.title()))
    

In [4]:
def save_to_excel(dataframe1,dataframe2):
    with pd.ExcelWriter(filename) as writer:
        dataframe1.to_excel(writer,sheet_name='User Goals')
        dataframe2.to_excel(writer,sheet_name='Data Log')
        writer.save()
        
def read_goals():
    global user_df
    global df
    global total_df
    if os.path.isfile(filename):
        user_df = pd.read_excel(filename,sheet_name='User Goals', index_col = 0)
        df = pd.read_excel(filename, sheet_name='Data Log', index_col = 0)
        total_df.append(df)
        name = user_df.index[0]
        goal = user_df['Goal'][0]
        started = user_df['Starting'][0]
        progress = user_df['Progress'][0]
        print('Welcome back %s.' % name)
        time.sleep(1)
        print('Let\'s see how your goal is going.')
        time.sleep(1)
        print('Your goal was %s gp.' % goal)
        time.sleep(1)
        print('You started with %s gp.' % started)
        time.sleep(1)
        print('You\'ve made %s gp.' % progress)
        print('Which leads you at a final cash stack of '+ str(started + progress) + ' gp.')
        
    else:
        print('No log data found. Let\'s make a new one.')
        name = input('What\'s your name?')
        goal = input('What\'s your gp goal?')
        started = input('How much are you starting with?')
        progress = 0
        user_df = pd.DataFrame({'Goal':[goal],'Starting':[started],'Progress':[progress]}, index = [name])
        df = pd.DataFrame({})
        with pd.ExcelWriter(filename) as writer:
            user_df.to_excel(writer,sheet_name='User Goals')
            df.to_excel(writer,sheet_name='Data Log')
            writer.save()
                    

In [22]:
total_df = []
columns = ['Name','Profit','Day','Amount','Method','Hours']
filename = 'RSgoal.xlsx'
user_df=None
df = None

In [ ]:
read_goals()
while True:
    print("What would you like to do?\n1.Add Data\n2.Edit data\n3.Delete data\n4.Check/Sort\n5.End ")
    answer= input("")
    if answer == '1':
        df = add_item()
        df = sort_by_date(df)
        save_to_excel(user_df,df)
    elif answer == '2':
        df = sort_item(df)
        print(df)
        df = edit_item(df)
        df = sort_by_date(df)
        save_to_excel(user_df,df)
    elif answer == '3':
        df = sort_item(df)
        print(df)
        df = delete_item(df)
        df = sort_by_date(df)
        save_to_excel(user_df,df)
        print(df)
    elif answer == '4':
        check_log(df)
    elif answer == '5':
        df = sort_by_date(df)
        save_to_excel(user_df,df)
        break

Welcome back iSlay4yu.
Let's see how your goal is going.
Your goal was 100000000 gp.
You started with 20000000 gp.
You've made 0 gp.
Which leads you at a final cash stack of 20000000 gp.
What would you like to do?
1.Add Data
2.Edit data
3.Delete data
4.Check/Sort
5.End 
4
                    Name   Profit  Amount        Day Method
0                Vorkath  5500000      40 2019-08-11   Boss
1                Vorkath  2610000      21 2019-08-13   Boss
2                guthans   109404      10 2019-08-12   Flip
3            book of law   200000       5 2019-08-12   Flip
4             Veracs set    47000       1 2019-08-12   Flip
5    Ancient dhide boots   280000       7 2019-08-12   Flip
6            Book of Law   405590       5 2019-08-13   Flip
7   Anguish Ornament Kit   115000       1 2019-08-13   Flip
8     Bandos dhide boots    40000       1 2019-08-13   Flip
9           Ranger tunic   219990       2 2019-08-13   Flip
10           book of law  2175000      15 2019-08-13   Flip
11     